In [1]:
!pip install opendatasets
!pip install keras.preprocessing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.5 MB/s eta 0:00:00


In [2]:
import opendatasets as od
import pandas as pd
import numpy as np
import os
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
od.download("https://www.kaggle.com/datasets/ananthu017/emotion-detection-fer?select=train")
od.download("https://www.kaggle.com/datasets/ananthu017/emotion-detection-fer?select=test")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: ghglhgkh
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/ananthu017/emotion-detection-fer


100%|██████████| 65.2M/65.2M [00:00<00:00, 1.63GB/s]

Skipping, found downloaded files in "./emotion-detection-fer" (use force=True to force download)


In [4]:
data_path = "/content/emotion-detection-fer"
data = os.listdir(data_path)
print(data)

['train', 'test']


In [5]:
for split in data:
    split_path = os.path.join(data_path, split)
    classes = os.listdir(split_path)
    print(f"{split} classes:", classes)

    counts = {cls: len(os.listdir(os.path.join(split_path, cls))) for cls in classes}
    print(f"{split} samples per class:", counts)

train classes: ['surprised', 'fearful', 'neutral', 'sad', 'disgusted', 'happy', 'angry']
train samples per class: {'surprised': 3171, 'fearful': 4097, 'neutral': 4965, 'sad': 4830, 'disgusted': 436, 'happy': 7215, 'angry': 3995}
test classes: ['surprised', 'fearful', 'neutral', 'sad', 'disgusted', 'happy', 'angry']
test samples per class: {'surprised': 831, 'fearful': 1024, 'neutral': 1233, 'sad': 1247, 'disgusted': 111, 'happy': 1774, 'angry': 958}


# Data normalization

In [6]:
train_data_generator = ImageDataGenerator(rescale=1./255)
test_data_generator = ImageDataGenerator(rescale=1./255)

# preprocessing train data

In [7]:
train_gen = train_data_generator.flow_from_directory(
    "/content/emotion-detection-fer/train",
    target_size=(48, 48),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical"
)

test_gen = test_data_generator.flow_from_directory(
    '/content/emotion-detection-fer/test',
    target_size=(48, 48),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical"
)


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


# Model archticture

In [8]:
# create model structure
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

# Train the neural network/model

In [10]:
emotion_model_info = emotion_model.fit(
        train_gen,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=test_gen,
        validation_steps=7178 // 64)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 30s 40ms/step - accuracy: 0.2397 - loss: 1.8344 - val_accuracy: 0.2990 - val_loss: 1.7525
Epoch 2/50
  1/448 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.3438 - loss: 1.6840

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


448/448 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.3438 - loss: 1.6840 - val_accuracy: 0.3012 - val_loss: 1.7517
Epoch 3/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 14s 30ms/step - accuracy: 0.3256 - loss: 1.6895 - val_accuracy: 0.4180 - val_loss: 1.5467
Epoch 4/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.4062 - loss: 1.5540 - val_accuracy: 0.4148 - val_loss: 1.5481
Epoch 5/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 13s 28ms/step - accuracy: 0.4047 - loss: 1.5418 - val_accuracy: 0.4452 - val_loss: 1.4495
Epoch 6/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.3750 - loss: 1.4280 - val_accuracy: 0.4485 - val_loss: 1.4492
Epoch 7/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 21s 46ms/step - accuracy: 0.4358 - loss: 1.4653 - val_accuracy: 0.4732 - val_loss: 1.3918
Epoch 8/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5625 - loss: 1.3076 - val_accuracy: 0.4738 - val_loss: 1.3911
Epoch 9/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 15s 32ms/step - accuracy: 0.4695 - loss: 1.3937 - val_accuracy: 0.494

# save model

In [12]:
# save trained model weight in .h5 file
emotion_model.save_weights('emotion_model.weights.h5')